# Getting Connected

The simplest way to try out a connection to the Neo4j server is via the console. Once you have started a local Neo4j server, open a new Python console and enter the following:

In [3]:
from py2neo import Graph
graph = Graph()

# Nodes & Relationships

Nodes and relationships are the fundamental building blocks of a Neo4j graph and both have a corresponding class in py2neo. Assuming we’ve already established a connection to the server (as above) let’s build a simple graph with two nodes and one relationship:

In [4]:
from py2neo import Node, Relationship
alice = Node("Person", name="Alice")
bob = Node("Person", name="Bob")
alice_knows_bob = Relationship(alice, "KNOWS", bob)
graph.create(alice_knows_bob)

Unauthorized: http://localhost:7474/db/data/

When first created, Node and Relationship objects exist only in the client; nothing has yet been written to the server. The Graph.create method shown above creates corresponding server-side objects and automatically binds each local object to its remote counterpart. 

Within py2neo, binding is the process of applying a URI to a client object thereby allowing future client-server synchonisation operations to occur.

# Pushing & Pulling

Client-server communication over REST can be chatty if not used carefully. Whenever possible, py2neo attempts to minimise the amount of chatter between the client and the server by batching and lazily retrieving data. Most read and write operations are explicit, allowing the Python application developer a high degree of control over network traffic.

To illustrate synchronisation, let’s give Alice and Bob an age property each. Longhand, this is done as follows:

In [3]:
alice.properties["age"] = 33
bob.properties["age"] = 44
alice.push()
bob.push()

Here, we add a new property to each of the two local nodes and push the changes in turn. This results in two separate HTTP calls being made to the server which can be seen more clearly with the debugging function, watch:

In [11]:
from py2neo import watch

watch("httpstream")
alice.push()

INFO:py2neo.batch:> Sending batch request with 2 jobs


> Sending batch request with 2 jobs


INFO:py2neo.batch:> {0} PUT node/0/properties {"age":33,"name":"Alice"}


> {0} PUT node/0/properties {"age":33,"name":"Alice"}


INFO:py2neo.batch:> {1} PUT node/0/labels ["Person"]


...

< Received batch response for 2 jobs


INFO:py2neo.batch:< {0} 204


< {0} 204


INFO:py2neo.batch:< {1} 204


< {1} 204


In [12]:
bob.push()

INFO:py2neo.batch:> Sending batch request with 2 jobs


> Sending batch request with 2 jobs


INFO:py2neo.batch:> {0} PUT node/1/properties {"age":44,"name":"Bob"}


> {0} PUT node/1/properties {"age":44,"name":"Bob"}


INFO:py2neo.batch:> {1} PUT node/1/labels ["Person"]


...

< Received batch response for 2 jobs


INFO:py2neo.batch:< {0} 204


< {0} 204


INFO:py2neo.batch:< {1} 204


< {1} 204


To squash these two separate push operations into one, we can use the Graph.push method instead:

In [13]:
graph.push(alice, bob)

INFO:py2neo.batch:> Sending batch request with 4 jobs


> Sending batch request with 4 jobs


INFO:py2neo.batch:> {0} PUT node/0/properties {"age":33,"name":"Alice"}


> {0} PUT node/0/properties {"age":33,"name":"Alice"}


INFO:py2neo.batch:> {1} PUT node/0/labels ["Person"]


...

< {1} 204


INFO:py2neo.batch:< {2} 204


< {2} 204


INFO:py2neo.batch:< {3} 204


< {3} 204


# Cypher

## Single Statements

Neo4j has a built-in data query and manipulation language called Cypher. To execute Cypher from within py2neo, simply use the cypher attribute of a Graph instance and call the execute method:

In [14]:
graph.cypher.execute("CREATE (c:Person {name:{N}}) RETURN c", {"N": "Carol"})

INFO:py2neo.cypher:begin


begin


INFO:py2neo.cypher:append u'CREATE (c:Person {name:{N}}) RETURN c' {'N': 'Carol'}


append u'CREATE (c:Person {name:{N}}) RETURN c' {'N': 'Carol'}


INFO:py2neo.cypher:commit


...

INFO:httpstream:< 200 OK [1158]


< 200 OK [1158]


INFO:py2neo.cypher:result [u'c'] 1


result [u'c'] 1


   | c                         
---+----------------------------
 1 | (n2:Person {name:"Carol"})

The object returned from this call is a RecordList which is rendered by default as a table of results. Each item in this list is a Record instance:

In [15]:
for record in graph.cypher.execute("CREATE (d:Person {name:'Dave'}) RETURN d"):
    print(record)

INFO:py2neo.cypher:begin


begin


INFO:py2neo.cypher:append u"CREATE (d:Person {name:'Dave'}) RETURN d" {}


append u"CREATE (d:Person {name:'Dave'}) RETURN d" {}


INFO:py2neo.cypher:commit


...

> POST http://localhost:7474/db/data/transaction/commit [119]
< 200 OK [1157]


INFO:httpstream:< 200 OK [1157]


< 200 OK [1157]


INFO:py2neo.cypher:result [u'd'] 1


result [u'd'] 1
 d                        
---------------------------
 (n3:Person {name:"Dave"})



A Record exposes its values through both named attributes and numeric indexes. Therefore, if a Cypher query returns a column called name, that column can be accessed through the record attribute called name:

In [16]:
for record in graph.cypher.execute("MATCH (p:Person) RETURN p.name AS name"):
    print(record.name)

INFO:py2neo.cypher:begin


begin


INFO:py2neo.cypher:append u'MATCH (p:Person) RETURN p.name AS name' {}


append u'MATCH (p:Person) RETURN p.name AS name' {}


INFO:py2neo.cypher:commit


...

> POST http://localhost:7474/db/data/transaction/commit [117]
< 200 OK [128]


INFO:httpstream:< 200 OK [128]


< 200 OK [128]


INFO:py2neo.cypher:result [u'name'] 4


result [u'name'] 4
Alice
Bob
Carol
Dave


Similarly, the first column returned can be accessed as column 0:

In [17]:
for record in graph.cypher.execute("MATCH (p:Person) RETURN p.name AS name"):
    print(record[0])

INFO:py2neo.cypher:begin


begin


INFO:py2neo.cypher:append u'MATCH (p:Person) RETURN p.name AS name' {}


append u'MATCH (p:Person) RETURN p.name AS name' {}


INFO:py2neo.cypher:commit


...

> POST http://localhost:7474/db/data/transaction/commit [117]
< 200 OK [128]


INFO:httpstream:< 200 OK [128]


< 200 OK [128]


INFO:py2neo.cypher:result [u'name'] 4


result [u'name'] 4
Alice
Bob
Carol
Dave


Transactions
http://py2neo.org/2.0/intro.html